#### IMPORT LIBRARIES

In [1]:
import numpy as np
import h5py as h5

import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable


import matplotlib.pyplot as plt
from importlib import reload, import_module

import glob
import os

import pdb
from PIL import Image as im
import _pickle as pickle
import cv2

from datetime import datetime

from functions import processLF, get_variable, get_numpy, psnr_1, sd_to_usd

#### DATASET PATH

In [2]:
if os.name == 'nt':
    dataset_file = r"C:\Users\mummu\Documents\Datasets\srinivasan\trainset\h5\8bit.h5"
    test_file    = r"C:\Users\mummu\Documents\Datasets\srinivasan\testset\h5\8bit.h5"
    model_file   = r"model\model.pt"
    network_file = r"network"
    img_dir      = r"C:\Users\mummu\Documents\Datasets\kalantari\testset\EXTRA"
    img_paper    = r"C:\Users\mummu\Documents\Datasets\kalantari\testset\PAPER"
    test_sri     = r"C:\Users\mummu\Documents\Datasets\srinivasan\testset"
    
        
elif os.name == 'posix':
    raise NotImplementedError

#### BASIC PARAMETERS

In [3]:
minibatch_size = 1
gamma_val      = 1.0
lfsize         = [372, 540, 7, 7]
batch_affine   = True

In [4]:
trans = transforms.ToTensor()
p = np.ndarray([1])
q = np.ndarray([1])

In [5]:
network_module = import_module(network_file)
reload(network_module)

Net = network_module.Net

net = Net((lfsize[0], lfsize[1]), minibatch_size, lfsize, batchAffine=batch_affine)
net.eval()

if torch.cuda.is_available():
    print('##converting network to cuda-enabled')
    net.cuda()

try:
    checkpoint = torch.load(model_file)
    
    net.load_state_dict(checkpoint['model'].state_dict())    
    print('Model successfully loaded.')
    
except:
    print('No model.')

##converting network to cuda-enabled
Model successfully loaded.


In [6]:
# To delete
#result = im.fromarray((get_numpy(corn[0,:3].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('corner1_f.png');
#result = im.fromarray((get_numpy(corn[0,3:6].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('corner2_f.png');
#result = im.fromarray((get_numpy(corn[0,6:9].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('corner3_f.png');
#result = im.fromarray((get_numpy(corn[0,9:].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('corner4_f.png');

In [7]:
def single_run(index, img_path = img_paper, img_name = 'Seahorse.png'):
            
    img = cv2.imread(os.path.join(img_path, img_name))
                    
    img = processLF(trans(img), lfsize, gamma_val)
    
    T = img[:, :, index[0], index[1], :].squeeze()
    corn = img[:, :, [0, -1, 0, -1], [0, 0, -1, -1], :].squeeze()
    
    Y, R, d = synthesizeView(corn, index)

    return T, Y, d

In [8]:
def synthesizeView(corn, index):
                
    p[0] = (index[0] - lfsize[2]//2)/(lfsize[2]//2)
    q[0] = (index[1] - lfsize[3]//2)/(lfsize[3]//2)
    
    corn = corn.permute(2,3,0,1).reshape(12,corn.shape[0],corn.shape[1])[None,:]

    with torch.no_grad():
        Y, R, d = net(get_variable(corn), get_variable(torch.from_numpy(p)), get_variable(torch.from_numpy(q)))
        
    return Y[0].permute(1,2,0), R[0].permute(1,2,0), d

In [9]:
#Y,R,d = single_run([3,3], img_name = 'Rock.png')
#plt.imshow(get_numpy(d[0,0]+4)/8, cmap = 'gray', vmin = 0, vmax = 1.0);

In [10]:
# Run for all examples in the folder and every perspectives
def run_all_examples(img_path):
        
    files = [file for file in os.listdir(img_path) if file.endswith(".png")]
    ps = np.ndarray((len(files),7,7))

    for fi in range(len(files)):

        file = files[fi]

        print("Current file {}: {}" .format(fi,file))

        img = cv2.imread(os.path.join(img_path, file))
        img = processLF(trans(img), lfsize, gamma_val)

        corn = img[:, :, [0, -1, 0, -1], [0, 0, -1, -1], :].squeeze()

        for i in range(7):
            for j in range(7):

                T = get_numpy(img[:, :, i, j, :])
                Y,R,d = synthesizeView(corn, [i, j])
                Y = get_numpy(Y)

                ps[fi,i,j] = psnr_1(sd_to_usd(T), sd_to_usd(Y))
        
        psnr_excl_corners = ps[fi].reshape(-1)[[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()
        
        print("Current PSNR: {}" .format(psnr_excl_corners))
    
    return ps
                
    

In [11]:
# With Bias and affine (B_BA_GN with gamma = 1.0)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)

date and time = 11/04/2020 10:47:24
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 31.562804051466912
Current file 1: Flower1.png
Current PSNR: 33.042409235669965
Current file 2: Flower2.png
Current PSNR: 33.05761230973208
Current file 3: IMG_1085_eslf.png
Current PSNR: 40.581017234640015
Current file 4: IMG_1086_eslf.png
Current PSNR: 42.147651114452394
Current file 5: IMG_1184_eslf.png
Current PSNR: 41.78923838356847
Current file 6: IMG_1187_eslf.png
Current PSNR: 40.946153835603255
Current file 7: IMG_1306_eslf.png
Current PSNR: 38.055590290320936
Current file 8: IMG_1312_eslf.png
Current PSNR: 44.52790418289266
Current file 9: IMG_1316_eslf.png
Current PSNR: 38.50927456990736
Current file 10: IMG_1317_eslf.png
Current PSNR: 36.71043618722326
Current file 11: IMG_1320_eslf.png
Current PSNR: 36.591555403960065
Current file 12: IMG_1321_eslf.png
Current PSNR: 43.58893152577145
Current file 13: IMG_1324_eslf.png
Current PSNR: 47.91847880548245
Current file 14: IMG_1325_eslf.png
Current PSNR: 40.85802420040675
Current file 15: IMG_1327_eslf.png
Current P

ValueError: cannot reshape array of size 1470 into shape (100,newaxis)

In [13]:
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

38.054580727478324

In [22]:
# With Bias and affine (B_BA_GN with gamma = 1.0)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(test_sri)
ps.reshape(100,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 11/04/2020 08:09:36
Current file 0: IMG_2227_eslf.png
Current PSNR: 36.789850293782514
Current file 1: IMG_2326_eslf.png
Current PSNR: 37.405323085714556
Current file 2: IMG_2459_eslf.png
Current PSNR: 35.29839595976429
Current file 3: IMG_2463_eslf.png
Current PSNR: 38.26815234235622
Current file 4: IMG_3408_eslf.png
Current PSNR: 36.30268143067899
Current file 5: IMG_3416_eslf.png
Current PSNR: 36.08595409639734
Current file 6: IMG_3683_eslf.png
Current PSNR: 37.28302131904665
Current file 7: IMG_3694_eslf.png
Current PSNR: 34.62650735983884
Current file 8: IMG_4312_eslf.png
Current PSNR: 39.12376352281682
Current file 9: IMG_4457_eslf.png
Current PSNR: 37.30387991918289
Current file 10: IMG_4492_eslf.png
Current PSNR: 38.86682391264303
Current file 11: IMG_4498_eslf.png
Current PSNR: 37.341038126530876
Current file 12: IMG_4501_eslf.png
Current PSNR: 35.2839519088774
Current file 13: IMG_4502_eslf.png
Current PSNR: 38.06227398247785
Current file 14: IMG_4512_eslf.png

38.03256347785122

In [11]:
# With Bias and affine (B_BA_GN with gamma = 1.0)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 11/04/2020 07:56:02
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 31.65331653148153
Current file 1: Flower1.png
Current PSNR: 33.063138167784835
Current file 2: Flower2.png
Current PSNR: 33.10115348909338
Current file 3: IMG_1085_eslf.png
Current PSNR: 40.54895287616024
Current file 4: IMG_1086_eslf.png
Current PSNR: 41.92977879786509
Current file 5: IMG_1184_eslf.png
Current PSNR: 41.8014564459335
Current file 6: IMG_1187_eslf.png
Current PSNR: 40.98899065166328
Current file 7: IMG_1306_eslf.png
Current PSNR: 37.97723081759351
Current file 8: IMG_1312_eslf.png
Current PSNR: 44.591589972772745
Current file 9: IMG_1316_eslf.png
Current PSNR: 38.53157326406664
Current file 10: IMG_1317_eslf.png
Current PSNR: 36.6087282748514
Current file 11: IMG_1320_eslf.png
Current PSNR: 36.70399844544188
Current file 12: IMG_1321_eslf.png
Current PSNR: 43.65319311343416
Current file 13: IMG_1324_eslf.png
Current PSNR: 48.06540431111889
Current file 14: IMG_1325_eslf.png
Current PSNR: 40.8826237245765
Current file 15: IMG_1327_eslf.png
Current PSNR: 37.

38.07157288734314

In [11]:
# With Bias and affine (B_BA_GN with gamma = 1.0)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 11/04/2020 03:07:07
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 31.65890939638865
Current file 1: Flower1.png
Current PSNR: 33.09178915155239
Current file 2: Flower2.png
Current PSNR: 33.0424867295445
Current file 3: IMG_1085_eslf.png
Current PSNR: 40.47743286002026
Current file 4: IMG_1086_eslf.png
Current PSNR: 42.20309284326313
Current file 5: IMG_1184_eslf.png
Current PSNR: 41.785172253737024
Current file 6: IMG_1187_eslf.png
Current PSNR: 41.03474595522944
Current file 7: IMG_1306_eslf.png
Current PSNR: 38.06988527088612
Current file 8: IMG_1312_eslf.png
Current PSNR: 44.61539989200505
Current file 9: IMG_1316_eslf.png
Current PSNR: 38.83022970045545
Current file 10: IMG_1317_eslf.png
Current PSNR: 37.08614632482467
Current file 11: IMG_1320_eslf.png
Current PSNR: 36.60296386114104
Current file 12: IMG_1321_eslf.png
Current PSNR: 43.72027472827506
Current file 13: IMG_1324_eslf.png
Current PSNR: 47.90157212542906
Current file 14: IMG_1325_eslf.png
Current PSNR: 40.996305911865655
Current file 15: IMG_1327_eslf.png
Current PSNR: 3

38.120094612618765

In [11]:
# With Bias and affine (B_BA_GN with gamma = 1.0)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 10/04/2020 18:05:01
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 31.671143208376034
Current file 1: Flower1.png
Current PSNR: 32.9986811813141
Current file 2: Flower2.png
Current PSNR: 33.013353950695475
Current file 3: IMG_1085_eslf.png
Current PSNR: 40.524578196422794
Current file 4: IMG_1086_eslf.png
Current PSNR: 42.24410899690316
Current file 5: IMG_1184_eslf.png
Current PSNR: 41.52694205242328
Current file 6: IMG_1187_eslf.png
Current PSNR: 40.81034569079054
Current file 7: IMG_1306_eslf.png
Current PSNR: 37.95514657201201
Current file 8: IMG_1312_eslf.png
Current PSNR: 44.31296690303726
Current file 9: IMG_1316_eslf.png
Current PSNR: 38.913208189796855
Current file 10: IMG_1317_eslf.png
Current PSNR: 37.182436352989036
Current file 11: IMG_1320_eslf.png
Current PSNR: 36.73491793856874
Current file 12: IMG_1321_eslf.png
Current PSNR: 43.33182485605262
Current file 13: IMG_1324_eslf.png
Current PSNR: 46.97070450465198
Current file 14: IMG_1325_eslf.png
Current PSNR: 40.654299076075006
Current file 15: IMG_1327_eslf.png
Current PSN

38.02972855610069

In [15]:
# With Bias and affine (B_BA_GN with single feature network calling)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 10/04/2020 17:45:01
Current file 0: Cars.png
Current PSNR: 35.652806428074086
Current file 1: Flower1.png
Current PSNR: 33.454072975005275
Current file 2: Flower2.png
Current PSNR: 33.49075513358508
Current file 3: IMG_1085_eslf.png
Current PSNR: 39.05267021857513
Current file 4: IMG_1086_eslf.png
Current PSNR: 38.07664322264795
Current file 5: IMG_1184_eslf.png
Current PSNR: 39.20468578312582
Current file 6: IMG_1187_eslf.png
Current PSNR: 37.98801502837993
Current file 7: IMG_1306_eslf.png
Current PSNR: 37.117862996762696
Current file 8: IMG_1312_eslf.png
Current PSNR: 40.27463605636852
Current file 9: IMG_1316_eslf.png
Current PSNR: 35.179398655929596
Current file 10: IMG_1317_eslf.png
Current PSNR: 33.96704974987906
Current file 11: IMG_1320_eslf.png
Current PSNR: 35.212496280140165
Current file 12: IMG_1321_eslf.png
Current PSNR: 39.64762908444187
Current file 13: IMG_1324_eslf.png
Current PSNR: 38.923744277489426
Current file 14: IMG_1325_eslf.png
Current PSNR: 35

36.51832997359718

In [11]:
# With Bias and affine (B_BA_GN with single feature network calling)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 10/04/2020 17:32:53
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 31.56458244234256
Current file 1: Flower1.png
Current PSNR: 32.89688643086002
Current file 2: Flower2.png
Current PSNR: 32.94188745427576
Current file 3: IMG_1085_eslf.png
Current PSNR: 40.386279504937725
Current file 4: IMG_1086_eslf.png
Current PSNR: 42.25482126767094
Current file 5: IMG_1184_eslf.png
Current PSNR: 41.60761296663459
Current file 6: IMG_1187_eslf.png
Current PSNR: 40.86447713174883
Current file 7: IMG_1306_eslf.png
Current PSNR: 38.03130162805674
Current file 8: IMG_1312_eslf.png
Current PSNR: 44.6457955327356
Current file 9: IMG_1316_eslf.png
Current PSNR: 38.570005289907066
Current file 10: IMG_1317_eslf.png
Current PSNR: 36.71357308391853
Current file 11: IMG_1320_eslf.png
Current PSNR: 36.454964561180596
Current file 12: IMG_1321_eslf.png
Current PSNR: 43.57230434742184
Current file 13: IMG_1324_eslf.png
Current PSNR: 47.82780775145554
Current file 14: IMG_1325_eslf.png
Current PSNR: 40.67158872010631
Current file 15: IMG_1327_eslf.png
Current PSNR: 

37.95892629521809

In [10]:
# With Bias and affine (B_BA_GN with single feature network calling)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 10/04/2020 14:25:48
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 31.382833399655865
Current file 1: Flower1.png
Current PSNR: 32.70619330794434
Current file 2: Flower2.png
Current PSNR: 32.701620368135266
Current file 3: IMG_1085_eslf.png
Current PSNR: 40.37764533724835
Current file 4: IMG_1086_eslf.png
Current PSNR: 41.93206592807629
Current file 5: IMG_1184_eslf.png
Current PSNR: 41.49288818707576
Current file 6: IMG_1187_eslf.png
Current PSNR: 40.89677507920642
Current file 7: IMG_1306_eslf.png
Current PSNR: 38.11147950682954
Current file 8: IMG_1312_eslf.png
Current PSNR: 44.74333617134318
Current file 9: IMG_1316_eslf.png
Current PSNR: 39.077544026964205
Current file 10: IMG_1317_eslf.png
Current PSNR: 37.02407207482338
Current file 11: IMG_1320_eslf.png
Current PSNR: 36.23815301480639
Current file 12: IMG_1321_eslf.png
Current PSNR: 43.79007948925034
Current file 13: IMG_1324_eslf.png
Current PSNR: 47.93220093968056
Current file 14: IMG_1325_eslf.png
Current PSNR: 40.61517268722942
Current file 15: IMG_1327_eslf.png
Current PSNR:

37.8480558001118

In [11]:
# With Bias and affine (B_BA_GN with single feature network calling)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 10/04/2020 13:33:07
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 35.76724332094976
Current file 1: Flower1.png
Current PSNR: 33.46543979507194
Current file 2: Flower2.png
Current PSNR: 33.47927281416884
Current file 3: IMG_1085_eslf.png
Current PSNR: 39.19788291756552
Current file 4: IMG_1086_eslf.png
Current PSNR: 38.08389193907625
Current file 5: IMG_1184_eslf.png
Current PSNR: 39.377564174781064
Current file 6: IMG_1187_eslf.png
Current PSNR: 38.062408789266904
Current file 7: IMG_1306_eslf.png
Current PSNR: 37.12659381424504
Current file 8: IMG_1312_eslf.png
Current PSNR: 40.30032709667823
Current file 9: IMG_1316_eslf.png
Current PSNR: 35.379946675898864
Current file 10: IMG_1317_eslf.png
Current PSNR: 34.08107729429988
Current file 11: IMG_1320_eslf.png
Current PSNR: 35.202522995073004
Current file 12: IMG_1321_eslf.png
Current PSNR: 39.7031997164854
Current file 13: IMG_1324_eslf.png
Current PSNR: 38.89631821497982
Current file 14: IMG_1325_eslf.png
Current PSNR: 35.44505456113558
Current file 15: IMG_1327_eslf.png
Current PSNR:

36.6076056903314

In [19]:
# With Bias and affine (B_BA_GN with single feature network calling)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 10/04/2020 12:42:41
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 35.64493148428115
Current file 1: Flower1.png
Current PSNR: 33.63051648775872
Current file 2: Flower2.png
Current PSNR: 33.71048545646876
Current file 3: IMG_1085_eslf.png
Current PSNR: 39.12364287787234
Current file 4: IMG_1086_eslf.png
Current PSNR: 37.98122921801012
Current file 5: IMG_1184_eslf.png
Current PSNR: 39.301321105326075
Current file 6: IMG_1187_eslf.png
Current PSNR: 37.96784322591248
Current file 7: IMG_1306_eslf.png
Current PSNR: 37.02452944099787
Current file 8: IMG_1312_eslf.png
Current PSNR: 40.18912108501506
Current file 9: IMG_1316_eslf.png
Current PSNR: 35.28353977030705
Current file 10: IMG_1317_eslf.png
Current PSNR: 34.047813826731456
Current file 11: IMG_1320_eslf.png
Current PSNR: 35.27558061110782
Current file 12: IMG_1321_eslf.png
Current PSNR: 39.52844771118457
Current file 13: IMG_1324_eslf.png
Current PSNR: 38.78993227571625
Current file 14: IMG_1325_eslf.png
Current PSNR: 35.45246393194656
Current file 15: IMG_1327_eslf.png
Current PSNR: 

36.55053119011702

In [10]:
# With Bias and affine (B_BA_GN with revisions)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

date and time = 10/04/2020 08:29:29
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Current PSNR: 33.13520376628959
Current file 1: Flower1.png
Current PSNR: 30.1110657388269
Current file 2: Flower2.png
Current PSNR: 30.59833208692981
Current file 3: IMG_1085_eslf.png
Current PSNR: 37.1816968138568
Current file 4: IMG_1086_eslf.png
Current PSNR: 36.18199048998766
Current file 5: IMG_1184_eslf.png
Current PSNR: 38.771833602830554
Current file 6: IMG_1187_eslf.png
Current PSNR: 36.63478834981254
Current file 7: IMG_1306_eslf.png
Current PSNR: 33.87481352093022
Current file 8: IMG_1312_eslf.png
Current PSNR: 37.95681786459273
Current file 9: IMG_1316_eslf.png
Current PSNR: 34.27404369413072
Current file 10: IMG_1317_eslf.png
Current PSNR: 33.8919324429245
Current file 11: IMG_1320_eslf.png
Current PSNR: 32.49117355975516
Current file 12: IMG_1321_eslf.png
Current PSNR: 38.06056278452347
Current file 13: IMG_1324_eslf.png
Current PSNR: 38.14786332865406
Current file 14: IMG_1325_eslf.png
Current PSNR: 34.21159526095776
Current file 15: IMG_1327_eslf.png
Current PSNR: 34.3

34.720297609631146

In [9]:
# With Bias and affine (B_BA_GN)
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
ps = run_all_examples(img_dir)
ps.reshape(30,-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()
# 36.63222244519973

date and time = 09/04/2020 14:53:53
Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 35.68765470750665
Current file 1: Flower1.png
Current PSNR: 33.56225479544764
Current file 2: Flower2.png
Current PSNR: 33.61957833053744
Current file 3: IMG_1085_eslf.png
Current PSNR: 39.210608995630395
Current file 4: IMG_1086_eslf.png
Current PSNR: 38.149722113747615
Current file 5: IMG_1184_eslf.png
Current PSNR: 39.323658365850605
Current file 6: IMG_1187_eslf.png
Current PSNR: 37.98113817872829
Current file 7: IMG_1306_eslf.png
Current PSNR: 37.1022254914154
Current file 8: IMG_1312_eslf.png
Current PSNR: 40.24351599186964
Current file 9: IMG_1316_eslf.png
Current PSNR: 35.356232072390384
Current file 10: IMG_1317_eslf.png
Current PSNR: 34.054658340164565
Current file 11: IMG_1320_eslf.png
Current PSNR: 35.276994996364195
Current file 12: IMG_1321_eslf.png
Current PSNR: 39.69661654219664
Current file 13: IMG_1324_eslf.png
Current PSNR: 38.88528536776065
Current file 14: IMG_1325_eslf.png
Current PSNR: 35.543576878304464
Current file 15: IMG_1327_eslf.png
Current PS

36.63222244519973